# Ingredient recommender system.
_More like this on [kachkach.com](http://kachkach.com)_

In this notebook, we will use the dataset to build an ingredient recommender system. We will go from the most basic (counting ingredient co-occurrences) to the slightly more elaborate (matrix factorization), also looking at a useful information theory metric called "Pointwise Mutual Information" (PMI).

## Data loading, imports.

In [8]:
import numpy as np
import pandas as pd

import os
print(os.listdir("../whats-cooking"))

['.ipynb_checkpoints', 'a-detailed-explanation-of-keras-embedding-layer.ipynb', 'exploring-food-recipes-using-word2vec.ipynb', 'food_list.csv', 'ingredient-recommender-system.ipynb', 'input.csv', 'sample_submission.csv.zip', 'test.json', 'test.json.zip', 'test_1.json', 'train.json', 'train.json.zip', 'train_1.json', 'Untitled.ipynb']


In [9]:
df = pd.concat([pd.read_json('train_1.json'), pd.read_json('test_1.json')], sort=True).reset_index()


In [10]:
# Lower-casing all ingredients.
df.ingredients = df.ingredients.apply(lambda ings : [ing.lower() for ing in ings])

In [11]:
df.sample(5)

,index,cuisine,id,ingredients
9626,9626,mexican,48709,"[neutral oil, black beans, chili powder, garli..."
24283,24283,indian,31846,"[tea bags, sweetened condensed milk, clove, fr..."
4577,4577,cajun_creole,36043,"[water, rice, stewed tomatoes, onions, johnson..."
42749,2975,NaN,35686,"[black beans, jalapeno chilies, lime wedges, s..."
47009,7235,NaN,17484,"[refried beans, flour tortillas, mexican chees..."


In [12]:
len(df)

49718

## Calculating ingredient co-occurrences.

We will start by calculating the number of recipes in which to ingredients occurred together. This intuitively gives us a measure of how common it is to see two ingredients mixed up, and is a good first attempt at building our ingredient recommender:

In [13]:
import itertools
# Example of what the itertools.combinations function does.
list(itertools.combinations(df.ingredients[0][:5], 3))

[('romaine lettuce', 'black olives', 'grape tomatoes'),
 ('romaine lettuce', 'black olives', 'garlic'),
 ('romaine lettuce', 'black olives', 'pepper'),
 ('romaine lettuce', 'grape tomatoes', 'garlic'),
 ('romaine lettuce', 'grape tomatoes', 'pepper'),
 ('romaine lettuce', 'garlic', 'pepper'),
 ('black olives', 'grape tomatoes', 'garlic'),
 ('black olives', 'grape tomatoes', 'pepper'),
 ('black olives', 'garlic', 'pepper'),
 ('grape tomatoes', 'garlic', 'pepper')]

In [14]:
# Calculating ingredient counts and co-occurrences in recipes.
from collections import Counter
cooc_counts = Counter()
ing_count  = Counter()
for ingredients in df.ingredients:
    for ing in ingredients:
        ing_count[ing] += 1
    for (ing_a, ing_b) in itertools.combinations(set(ingredients), 2):
        # NOTE: just making sure we added pairs in a consistent order (a < b); you can also add both (a,b) and (b,a) if you want.
        if ing_a > ing_b:
            ing_a, ing_b = ing_b, ing_a
        cooc_counts[(ing_a, ing_b)] += 1

In [15]:
cooc_df = pd.DataFrame(((ing_a, ing_b, ing_count[ing_a], ing_count[ing_b], cooc) for (ing_a, ing_b), cooc in cooc_counts.items()), columns=['a', 'b', 'a_count', 'b_count', 'cooc'])
cooc_df.sample(10)

,a,b,a_count,b_count,cooc
346185,chopped pecans,sorghum,347,5,1
433805,country gravy,salt,2,22534,1
46837,apricot nectar,pineapple juice,10,107,2
307140,all-purpose flour,yellow lentils,5816,16,2
482236,extra large eggs,vegetable broth,39,506,1
370,sugar,vanilla extract,8064,1626,813
255558,clove,pink lentil,768,1,1
403385,garlic,pork stock,9171,10,3
184415,lemon,veal shanks,1556,32,3
398331,jumbo pasta shells,large egg yolks,50,723,1


In [16]:
cooc_df[cooc_df.a == 'chillies'].sort_values('cooc', ascending=False).head(10)

,a,b,a_count,b_count,cooc
3197,chillies,salt,148,22534,67
3191,chillies,onions,148,10008,63
3139,chillies,garlic,148,9171,59
3128,chillies,ginger,148,2190,47
58920,chillies,garam masala,148,1179,37
19361,chillies,vegetable oil,148,5516,36
3177,chillies,water,148,9293,32
21199,chillies,coriander,148,560,31
80998,chillies,garlic cloves,148,7772,31
3185,chillies,tumeric,148,933,31


Notice how the elements with the highest co-occurrence count are not necessarily very similar, just overall very popular ingredients.

## Pointwise Mutual Information

We surfaced a number of issues with using the raw number of co-occurrences, mainly the fact that this measure is highly biased by the popularity of either items. One better metric when looking at correlation is [PMI](https://en.wikipedia.org/wiki/Pointwise_mutual_information), and its formula goes something like this:****

$$PMI(A, B) = log \frac{P(A, B)}{P(A) \times P(B)}$$

In [17]:
# We calculate P(A), P(B) and P(A, B) and PMI(A, B) from the previous df.
# P(A) is counts(A) / num_recipes
# P(A, B) is coocs(A, B) / sum(coocs)
p_a = cooc_df.a_count / sum(ing_count.values())
p_b = cooc_df.b_count / len(ing_count.values())
p_a_b = cooc_df.cooc / cooc_df.cooc.sum()
cooc_df['pmi'] = np.log(p_a_b / (p_a * p_b))

Simply ordering by PMI values givers us an extra argument for removing rare ingredients: they have a noisy PMI value, since we didn't see them in enough contexts to give enough support to the "lift ratio" that PMI is. 
This also shows that we really should be using unigrams/bigrams of the ingredient's textual description, instead of using the full ingredient name, as "vinegar" should be treated similarly to "red vinegar", but the current approach treats these two ingredients as being totally different.

In [18]:
cooc_df.sort_values('pmi', ascending=False).head(10)

,a,b,a_count,b_count,cooc,pmi
242530,jimmy dean pork sausage,pam no-stick cooking spray,1,1,1,7.113655
277036,flora original,knorr chicken stock cubes,1,1,1,7.113655
504843,dried cilantro leaves,vegan milk substitute,1,1,1,7.113655
163029,conimex wok olie,conimex woksaus specials vietnamese gember kno...,1,1,1,7.113655
246334,kampyo,katsuo dashi,1,1,1,7.113655
310794,bagel chips,jamaican jerk,1,1,1,7.113655
266449,gluten-free oat,psyllium husks,1,1,1,7.113655
430322,kraft shredded colby & monterey jack cheese,kraft zesty lime vinaigrette dressing,1,1,1,7.113655
457391,peach sorbet,raspberry sherbet,1,1,1,7.113655
233499,mazolaâ® chicken flavor bouillon powder,spice islands bay leaves,1,1,1,7.113655


I would go all in when it comes to filtering low frequency ingredients.
For low values of `min_count`, we get some very peculiar pairs which are likely due to the recipes being from the same website that has some advertising partnership with the brands mentionned.

In [19]:
min_count = 5
cooc_df[(cooc_df.a_count >= min_count) & (cooc_df.b_count >= min_count)].sort_values('pmi', ascending=False).head(20)

,a,b,a_count,b_count,cooc,pmi
116910,gourmet garden garlic paste,pompeian canola oil and extra virgin olive oil,5,6,5,5.321896
326841,buttermilk cornbread,muffin mix,6,5,4,5.098752
69411,brats,knockwurst,8,5,4,4.811070
116922,gourmet garden garlic paste,johnsonville andouille,5,10,4,4.587927
415809,kraft shredded cheddar cheese,taco bellâ® thick & chunky mild salsa,5,5,2,4.587927
134550,sazon seasoning,sofrito,5,5,2,4.587927
172102,herdez salsa casera,herdez salsa verde,5,8,3,4.523388
207998,black treacle,porridge oats,8,5,3,4.523388
344323,cola soft drink,cooked bone in ham,6,5,2,4.405605
100506,chinese rose wine,maltose,5,9,3,4.405605


With values around 40 or 50, we start to see real correlations appear:

In [20]:
min_count = 30
cooc_df[(cooc_df.a_count >= min_count) & (cooc_df.b_count >= min_count)].sort_values('pmi', ascending=False).head(20)

,a,b,a_count,b_count,cooc,pmi
35423,mexican chocolate,pasilla chiles,41,38,12,2.247404
171963,gari,wasabi,57,34,13,2.109193
69471,juniper berries,sauerkraut,30,40,7,1.969489
14680,brown cardamom,green cardamom,55,107,34,1.959854
217866,cilantro root,shrimp paste,30,67,11,1.905661
62578,bonito flakes,konbu,52,85,22,1.810803
84129,asafoetida powder,fresh curry leaves,45,91,20,1.791866
62639,dried bonito flakes,konbu,49,85,20,1.774916
109240,sushi rice,wasabi,97,34,15,1.720634
70809,niã§oise olives,tuna steaks,30,52,7,1.707125


We can also look at the pairs with the lowest PMI. We filter out pairs with only one co-occurrence, as those seem like outliers.
Notice also that we cannot use negative values to acertain how unlikely a combination is: if a pair is so unlikely, we will have little to no support for it (small or null number of coocs). Because of this, we usually drop the negative PMI values.

In [21]:
min_count = 30
cooc_df[(cooc_df.a_count >= min_count) & (cooc_df.b_count >= min_count) & (cooc_df.cooc > 1)].sort_values('pmi', ascending=True).head(20)

,a,b,a_count,b_count,cooc,pmi
410819,green onions,vanilla extract,3817,1626,2,-7.834296
410911,pepper,vanilla extract,5508,1626,3,-7.795567
272149,dried oregano,ginger,2163,2190,2,-7.564106
271488,fresh ginger,grated parmesan cheese,1846,2367,2,-7.483352
311134,garlic,powdered sugar,9171,616,3,-7.334781
124847,chili powder,dry white wine,2519,1492,2,-7.332687
478057,cucumber,unsalted butter,970,3474,2,-7.223555
417758,minced garlic,vanilla extract,2001,1626,2,-7.188478
105264,grated parmesan cheese,soy sauce,2367,4120,6,-7.187572
417526,soy sauce,whipping cream,4120,779,2,-7.174817


## Matrix Factorization

There's an issue with the approach we used previously: we are only leveraging direct correlations between ingredients (say, the fact that there are 15 recipes with both `sushi rice` and `wasabi`) and not using all the knowledge that can be extracted from more subtle correlations, which is particularly useful for less popular items.
Example: There might not be many recipes between some particular type of Mexican pepper and `corn tortillas`, but since that pepper appears with other ingredients similar to a `tortilla`, we would expect it to be similar to `corn tortillas`.

One solution to this *sparseness* problem (the fact that most pairs of ingredients have little to no co-occurrences) is to use Matrix Factorization. I won't go into the details of the linear algebra behind this technique (I would recommend checking the [matrix factorization wikipedia page](https://en.wikipedia.org/wiki/Matrix_decomposition)), but here goes a simple illustration of how we will use it:

- First, we create a matrix where rows and columns represent ingredients, and the values are the PMI of a pair of ingredients. (you might also use a binary co-occurrence signal, e.g 1 if there's any recipe with both ingredients, 0 otherwise; or use the raw number of co-occurrences, but PMI makes more sense in our case)
- We factorize this matrix: You can think of it as "compressing" our matrix from a large but sparse NxN matrix, where N is the number of ingredients, to a smaller but dense NxK matrix, where K is a number that we choose (hereset to 120 as it gave decent results).

Matrix factorization is helpful because it _generalizes_ the knowledge we have about ingredients, and removes noise and redundancies in the data. The output of this step is a vector representing each ingredient, vectors that we can compare to each other using various similarity metrics. Given that in this case, the most popular ingredients will have larger vectors, we prefer [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) since it is not biased by a vector's norm.

In [22]:
from scipy.sparse import csr_matrix
data_df = cooc_df[cooc_df.pmi > 0].copy()
# Since the matrix is symetric, we add the same values for (b,a) as we have for (a,b)
data_df_t = data_df.copy()
data_df.a, data_df.b = data_df.b, data_df.a
data_df = pd.concat([data_df, data_df_t])

rows_idx, row_keys = pd.factorize(data_df.a)
cols_idx, col_keys = pd.factorize(data_df.b)
values = data_df.pmi

matrix = csr_matrix((values, (rows_idx, cols_idx)))
key_to_row = {key: idx for idx, key in enumerate(row_keys)}

In [23]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(200)
factors = svd.fit_transform(matrix)

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
def most_similar(ingredient, topn=10):
    if ingredient not in key_to_row:
        print("Unknown ingredient.")
    factor = factors[key_to_row[ingredient]]
    cosines = cosine_similarity([factor], factors)[0]
    indices = cosines.argsort()[::-1][:topn + 1]
    keys = [row_keys[idx] for idx in indices if idx != key_to_row[ingredient]]
    return keys, cosines[indices]

def display_most_similar(ingredient, topn=10):
    print("- Most similar to '{}'".format(ingredient))
    for similar_ing, score in zip(*most_similar(ingredient, topn)):
        print("  . {} : {:.2f}".format(similar_ing, score))    

And tada, we're done! Here are some examples of recommendations generated by our model:

In [25]:
display_most_similar('chile powder')

- Most similar to 'chile powder'
  . tapatio hot sauce : 1.00
  . cottage cheese : 0.98
  . chipotle : 0.98
  . ground turkey : 0.98
  . chimichurri : 0.98
  . full fat sour cream : 0.98
  . flank steak : 0.97
  . mexican oregano : 0.97
  . kraft sharp cheddar cheese : 0.96
  . slaw : 0.96


In [26]:
display_most_similar('harissa')

- Most similar to 'harissa'
  . couscous : 1.00
  . dried mint flakes : 0.98
  . chapati : 0.98
  . pitas : 0.97
  . chickpea flour : 0.96
  . quinoa : 0.95
  . bulgur wheat : 0.95
  . roti : 0.95
  . gouda : 0.93
  . feta cheese crumbles : 0.91


In [27]:
display_most_similar('noodles')

- Most similar to 'noodles'
  . chili sauce : 1.00
  . cumin : 0.83
  . goat cheese : 0.83
  . eggplant : 0.83
  . polenta : 0.83
  . ground turmeric : 0.83
  . paneer : 0.82
  . bread : 0.82
  . sparkling lemonade : 0.82
  . prawns : 0.82


In [43]:
display_most_similar('broccoli')

- Most similar to 'broccoli'
  . ground beef : 1.00
  . veggie patties : 1.00
  . italian seasoned breadcrumbs : 1.00
  . shortening : 1.00
  . bread crumbs : 1.00
  . knorrâ® beef bouillon : 1.00
  . cabbage : 0.99
  . ackee : 0.99
  . squash : 0.99
  . grilled chicken strips : 0.98


In [29]:
display_most_similar('vanilla')

- Most similar to 'vanilla'
  . poppy seed filling : 1.00
  . pandan essence : 0.99
  . raw milk : 0.97
  . hot cross buns : 0.95
  . sugar pearls : 0.91
  . cassis : 0.89
  . oil of orange : 0.88
  . instant oats : 0.66
  . refined sugar : 0.57
  . mint extract : 0.56


In [30]:
display_most_similar('whipped cream')

- Most similar to 'whipped cream'
  . hot fudge topping : 1.00
  . amarena cherries : 1.00
  . gingersnap cookies : 1.00
  . fast rising yeast : 1.00
  . meyer lemon peel : 1.00
  . ibarra : 1.00
  . blackstrap molasses : 0.99
  . chestnut purã©e : 0.99
  . gingersnap cookie crumbs : 0.98
  . marshmallow vodka : 0.96


In [35]:
display_most_similar('buffalo mozarella')

- Most similar to 'buffalo mozarella'
  . hot pepperoni : 1.00
  . stonefire italian thin pizza crust : 1.00
  . smoked gouda : 0.97
  . new york style panetiniâ® toasts : 0.92
  . jarlsberg : 0.91
  . sweet yellow corn : 0.90
  . crumbled cornbread : 0.89
  . cheese ravioli : 0.84
  . raclette : 0.81
  . smoked cheddar cheese : 0.79


In [32]:
display_most_similar('beef')

- Most similar to 'beef'
  . beef broth : 1.00
  . cabbage : 0.84
  . italian seasoned breadcrumbs : 0.77
  . broccoli : 0.76
  . bread crumbs : 0.76
  . ground beef : 0.76
  . shortening : 0.76
  . veggie patties : 0.75
  . jamaica : 0.75
  . ackee : 0.75


In [46]:
display_most_similar('shrimp')

- Most similar to 'shrimp'
  . soy marinade : 1.00
  . chinese hot mustard : 0.67
  . nappa cabbage : 0.67
  . forest mushroom : 0.63
  . fillet of beef : 0.63
  . kinchay : 0.61
  . mae ploy sweet chili sauce : 0.61
  . reduced sodium teriyaki sauce : 0.61
  . lop chong : 0.61
  . black bean sauce with garlic : 0.61
